#Tarea 4. Riesgo de Crédito

> Nombre: Sánchez Rodríguez Ursula Vanessa

> Correo: vanecdp@ciencias.unam.mx

> No. de cuenta: 316039161



In [ ]:
%load_ext rpy2.ipython
!rm -rf Riesgos-Financieros
!git clone "https://github.com/vanecdp/Riesgos-Financieros-2022-2"

In [ ]:
%%R
#require(quantmod)
install.packages("quantmod")
library(quantmod)
#require(data.table)
install.packages("data.table")
library(data.table)
#require("PerformanceAnalytics")
install.packages("PerformanceAnalytics")
library("PerformanceAnalytics")

install.packages("Deriv")
library(Deriv)

install.packages("ResourceSelection")
library(ResourceSelection)
library(dplyr)
install.packages("ggplot2")
library(ggplot2)

install.packages("quantmod")
install.packages("curl")
install.packages("data.table")
install.packages("RCurl")
install.packages("bitops")
install.packages("setInternet2")
install.packages("rootSolve")
install.packages("xml2")
library(rootSolve)
library(quantmod)
library(RCurl)
library(data.table)
library(xml2)
require(jsonlite)

con = gzcon(url('https://github.com/systematicinvestor/SIT/raw/master/sit.gz', 'rb'))
source(con)
close(con)

## Ejercicio 1


Usando la tabla del archivo “Ejer anal discr.xls” (hoja base) que está en la página del curso, encontrar un discriminante lineal con las variables más significativas (pueden hacer uso de la hoja frecuencia del archivo
“logit y disc datos.xls ”o bien maximizando la $\lambda$ o minimizando la $F$ (igualdad de medias)), pueden aplicar también el archivo “andisc.m” para hacer el análisis, obviamente entregar el vector, la tabla de contingencia a priori/discriminante, valor de $\lambda$, valor de $F$ e interpretar el resultado.

**Solución.**

Cargamos la base que contiene las variables predictoras y a la variable de clasificación que en este caso es la de Incumplimiento.

In [ ]:
%%R
#Cargamos base
base <- read.csv("Riesgos-Financieros-2022-2/Tarea 4/Base.csv", header=TRUE, encoding = "UTF-8")

In [ ]:
%%R
head(base)

Haciendo varias pruebas se observó que la $\lambda$ se maximiza cuando crece el número de variables entonces definiremos la matriz que contiene los valores de las 14 variables como `m`.

In [ ]:
%%R
m <- base[,c(3:14)]
head(m)

Y definiremos las matrices de covarianzas y correlaciones.

In [ ]:
%%R
#Matriz de Varianzas y Covarianzas
Var_Cov <- cov(m)

#Clientes buenos
n1 <- 126
n2 <- 121

#Matriz de Correlación
Corr <- cor(m) 


Luego calcularemos las matrices con las medias de cada variable, donde `Medias_x_B` es para los clientes buenos y `Medias_x_M` para los clientes malos.

In [ ]:
%%R
#Matriz medias
Medias_x_B <- colMeans(m[1:126,])
Medias_x_M <- colMeans(m[127:247,])

Ahora, definimos las matrices *B*, *W*, *T* y el vector a, con el cual nos apoyaremos para definir el discriminante.

In [ ]:
%%R
#Matriz B
B <- (n1*n2)/sum(n1,n2)*as.matrix(Medias_x_B - Medias_x_M) %*% t(as.matrix(Medias_x_B - Medias_x_M))

#Matriz W
var_cov_b <- n1/(n1-1)*n1*cov(m[1:126,])
var_cov_m <- n2/(n2-1)*n2*cov(m[127:247,])
W <- var_cov_b+ var_cov_m
W_inv <- solve(W)
T <- B+W

#Vector a
a <- W_inv %*% as.matrix(Medias_x_B - Medias_x_M)


Finalmente, definimos el Discriminante Lineal de Fisher.

In [ ]:
%%R
#Discriminante Lineal de Fisher
discriminante_F <- matrix(0,dim(m)[1])

for (i in 1:dim(m)[1]){
    if(as.matrix((m[i,]- 1/2*(Medias_x_B+ Medias_x_M))) %*% as.matrix(a)>0){
        discriminante_F[i] <- 0
    }
    else{
        discriminante_F[i] <- 1
    }
}

Con la cual se obtiene el siguiente valor de $\lambda$.

In [ ]:
%%R
lambda <- ((t(as.matrix(a)) %*% B) %*% as.matrix(a))/((t(as.matrix(a)) %*% W) %*% as.matrix(a))
lambda

Así, en la siguiente tabla de contingencia podemos observar la comparación de la clasificación a priori con el discriminante lineal.

In [ ]:
%%R
#Tabla de contingencia
tabla_comp <- table(base$Incump,discriminante_F)
colnames(tabla_comp) <- c("Buenos", "Malos")
rownames(tabla_comp) <- c("Buenos", "Malos")
names(dimnames(tabla_comp)) <- c("Priori", "Discriminante de Fisher")
tabla_comp

Es decir, de los 126 clientes buenos, se clasificaron adecuadamente a 125 y mal a 1, por otro lado, de los 121 clientes malos, todos fueron clasificados adecuadamente.

In [ ]:
%%R
lambda_Wilks <- det(W)/det(T)
lambda_Wilks

In [ ]:
%%R
# F (valor observado)
F_obs <- (((t(as.matrix(a)) %*% B) %*% as.matrix(a))/count(Medias_x_B))/(((t(as.matrix(a)) %*% W) %*% as.matrix(a))/(sum(n1,n2)-count(Medias_x_B)-1))
F_obs

In [ ]:
%%R
alpha=0.05
F_critico <- qf(alpha, count(Medias_x_B), sum(n1,n2)-count(Medias_x_B)-1, lower.tail=F)
F_critico

In [ ]:
%%R
# p-value
p_value <- 1-  pf(F_obs, count(Medias_x_B), sum(n1,n2)-count(Medias_x_B)-1)
p_value


Dado que el p-value es 0 rechazamos la hipótesis nula de que las medias en los grupos son iguales.

## Ejercicio 3

Usando la tabla del archivo “Ejer anal discr.xls” (hoja base) que está en la página del curso, ajustar una regresión logística con las variables más significativas (pueden hacer uso de la hoja frecuencia del
archivo “logit y disc datos.xls” y utilizando la prueba de Wald para cada variable), pueden usar el archivo “logitprobit.ipynb” para hacer el análisis, obviamente entregar los coeficientes de regresión, la gráfica de datos observados y ajustados, el nivel de significancia (Devianza) del modelo, es decir la prueba de hipótesis de que la Devianza se distribuye como una $\chi^2_{n−(m+1)}$, y la interpretación de las variables.

**Solución.**

In [ ]:
%%R
head(base)

En este caso la variable *Incump* es la variable de respuesta que representa en Cumplimiento de cada cliente, toma los valores 0 y 1 (Cumplido=0, Incumplido=1).

Para medir la significancia las variable ajustaremos un modelo logístico usando como variable predictora a cada una de las variables, recordando que un p-value pequeño representa que el parámetro es estadísticamente distinto de cero.

In [ ]:
%%R
summary(glm(Incump ~Liquid,data=base, family=binomial(link="logit")))

Para la variable *Liquid* podemos observar que el p-value es muy pequeño por lo que resulta significativa, y la significancia impacta en el AIC, sabiendo que un AIC pequeño representa un buen ajuste por lo que para las demás variables nos fijaremos en este valor y nos quedaremos con las del AIC más pequeño.

In [ ]:
%%R
glm(Incump ~Apalanc,data=base, family=binomial(link="logit"))

In [ ]:
%%R
glm(Incump ~AñosCl,data=base, family=binomial(link="logit"))

In [ ]:
%%R
glm(Incump ~Rentab,data=base, family=binomial(link="logit"))

In [ ]:
%%R
glm(Incump ~Bancos,data=base, family=binomial(link="logit"))

In [ ]:
%%R
glm(Incump ~PCP_CT,data=base, family=binomial(link="logit"))

In [ ]:
%%R
glm(Incump ~Gop_SD ,data=base, family=binomial(link="logit"))

In [ ]:
%%R
glm(Incump ~RetrTMP,data=base, family=binomial(link="logit"))

In [ ]:
%%R
glm(Incump ~PLP_PT ,data=base, family=binomial(link="logit"))

In [ ]:
%%R
glm(Incump ~IFAudit,data=base, family=binomial(link="logit"))

In [ ]:
%%R
glm(Incump ~UO_VN,data=base, family=binomial(link="logit"))

In [ ]:
%%R
glm(Incump ~AñoOper,data=base, family=binomial(link="logit"))

Comparando los AIC que proporciona cada variable notamos que *Gop_SD*, *PCP_CT* y *Apalanc* son las que presentan un AIC más pequeño, es decir, son las más significativas, así que ajustaremos un modelo con estas variables.

In [ ]:
%%R
#Modelo de Rgresión Lineal
modelo1 <- glm(Incump ~  Apalanc + PCP_CT+ Gop_SD,data=base, family=binomial(link="logit"))
summary(modelo1)

Hemos obtenido los siguientes parámetros:

$$\hat{\beta_0}=-0.4715$$
$$\hat{\beta_1}= 5.5481 $$
$$\hat{\beta_2}= 8.2670 $$
$$\hat{\beta_3}= -16.8367 $$

El modelo tiene como covariables a variables continuas de las cuales las primeras dos son continuas, por lo que tenemos tantos patrones de covariables como sujetos por lo que considerar las estadísticas $\chi^2$ de Pearson y la Devianza (D) es inadecuado para una prueba de Bondad de Ajuste. Entonces, se utilizarás la Estadística de Hosmer-Lemeshow, que propone una solución cuando se tiene un patrón de covariables muy grande, a través de una prueba de bondad de ajuste basada en las probabilidades estimadas por el modelo, agrupando a los individuos por sus probabilidades estimadas, la recomendación es hacer 10 grupos.

In [ ]:
%%R
#Prueba de Hosmer-Lemshow
(HLtest<- hoslem.test(modelo1$y,fitted(modelo1), g = 10))

El p-value es igual a 1 por lo que considerando un nivel de significancia de $\alpha= 0.05$ concluimos que el modelo ajusta muy bien a los datos.

Con lo anterior podemos hacer una comparación de los valores ajustados versus los observados.

In [ ]:
%%R
cbind(HLtest$observed,HLtest$expected)

La variable $y_0$  representa el número de clientes buenos (Incump=0) y $y_1$  la cantidad de clientes malos (Incump=1) en el rango de probabilidades que les corresponde, mientras que los valores a la derecha son los ajustados por el modelo. Y podemos observar que el ajuste es bueno.

In [ ]:
%%R
# Plot Ajustados vs Observados
df <- data.frame(x=base$Cliente,y1=base$Incump,y2=modelo1$fitted.values)

colors <- c("Ajustados" = "#C80A5E", "Observados" = "#495AAB")
ggplot(df, aes(x=x)) + 
    geom_point(aes(y=y2, color="Ajustados"))+ 
    geom_point(aes(y=y1, color="Observados")) + 
    labs(x="Cliente", y="Incumplimiento", color = "Legend") +
    scale_color_manual(values = colors)


Como puede observarse los valores ajustados son continuos pero la variable de respuesta es binaria, por lo que podemos definir un punto de corte para realizar una adecuada clasificación y predicción. Si elegimos un punto de corte $c=0.5$ tenemos lo siguiente

In [ ]:
%%R
pred_mod<-ifelse(modelo1$fitted.values<=0.5,0,1)
tabla_cont <- table(base$Incump,pred_mod)
colnames(tabla_cont) <- c("Buenos", "Malos")
rownames(tabla_cont) <- c("Buenos", "Malos")
tabla_cont

La tabla anterior nos dice que de 126 clientes buenos el modelo clasificó correctamente 125 y 1 mal, y de 121 clientes malos el modelo clasificó adecuadamente a todos los individuos.

Por lo tanto, usando un modelo de regresión logística con predictores:
* Apalancamiento 
* Pasivo corto plazo a capital de trabajo
* Generación Operativa

tenemos un buen poder predictivo para clasificar a los clientes.

## Ejercicio 4

Utilizando información pública de la acción que ustedes decidan e información de su balance para obtener $K_0$ (buscar en la página de la empresa que decidan analizar), calcular $D$, $\sigma_K$, para $T = 1$ año, y probalilidad de incumplimiento al tiempo $T$ bajo el modelo de Merton. Calcular volatilidad anual de $\sigma_A$ con la historia de acciones, utilizar $r$ como el valor del cete más reciente, pueden utilizar el archivo de colab *“MertonKMVR.ipynb”*.

**Solución.**


La acción con la que se trabajará es de Mercado Libre (MELI).

In [ ]:
%%R
Symbols<-c  ("MELI")

start_date=Sys.Date()-3660 #fecha inicial

#Creación del objeto para guardar los datos
dataEnv<-new.env()

#obtener los datos
getSymbols.yahoo(Symbols,env=dataEnv,from=start_date)

#limpiarlos, alinearnos y quedarnos con el precio de cierre 

#muestra de datos
#Nos quedamos con los precios
stock_prices = na.omit(dataEnv$`MELI`)
stock_prices=stock_prices[,4] #Close
tail(stock_prices[,])

In [ ]:
%%R
#PARA CARGAR EL NÚMERO DE ACCIONES EN VENTA

getQuote <- function(ticks) {
  qRoot <- "https://query1.finance.yahoo.com/v7/finance/quote?fields=symbol,longName,regularMarketPrice,regularMarketChange,regularMarketTime,sharesOutstanding,bookValue&formatted=false&symbols="
  z <- fromJSON(paste(qRoot, paste(ticks, collapse=","), sep=""))
  z <- z$quoteResponse$result[,c("symbol", "regularMarketTime", "regularMarketPrice", "regularMarketChange", "longName", "sharesOutstanding", "bookValue" )]
  row.names(z) <- z$symbol
  z$symbol <- NULL
  names(z) <- c("Time", "Price", "Change", "Name", "Numeroacciones", "valorlibros")
  z$Time <- as.POSIXct(z$Time, origin = '1970-01-01 00:00:00')
  return(z)
}
ticks=Symbols
stqt=getQuote(Symbols)

#PARA CARGAR LOS ESTADOS FINANCIEROS Y FLUJOS DE EFECTIVO
url <- "https://finance.yahoo.com/quote/"
scrapy_stocks <- function(stock){
  if ("rvest" %in% installed.packages()) {
    library(rvest)
  }else{
    install.packages("rvest")
    library(rvest)
  }
  for (i in 1:length(stock)) {
    tryCatch(
      {
        # Income Statement
        # financial <- paste0(url,stock[i],"/financials?p=",stock[i])
        # wahis.session <- html_session(financial)                                
        # p <-    wahis.session %>% 
        #   html_nodes(xpath = '//*[@id="Col1-1-Financials-Proxy"]/section/div[4]/div[1]/div[1]/div[2]') %>%
        #   .[[1]] %>% html_children() %>% html_nodes("div")
        # suppressWarnings(Values <- p[sapply(p, function(x){length(x %>% html_attrs())}) == 3] %>% 
        #                    html_text() %>% gsub(pattern = ",",replacement = "") %>% 
        #                    gsub(pattern = "-",replacement = 0) %>% as.numeric()) 
        # Headers <- wahis.session %>% 
        #   html_nodes(xpath = '//*[@id="Col1-1-Financials-Proxy"]/section/div[4]/div[1]/div[1]') %>% 
        #   .[[1]] %>% html_nodes("span") %>% html_text() %>% head()
        # Concepts <- p[sapply(p, function(x){length(x %>% html_attrs())}) == 2] %>% html_text() %>% .[seq(2,88,4)]
        # IS <- data.frame(Breakdown = Concepts[-c(4,16,19)], matrix(Values[!is.na(Values)], ncol = 5, byrow = T))
        # names(IS) <- Headers
        # temp1 <- IS
        
        # Balance Sheet
        bsheet <- paste0(url,stock[i],"/balance-sheet?p=",stock[i])
        wahis.session <- html_session(bsheet)
        p <-    wahis.session %>% 
          html_nodes(xpath = '//*[@id="Col1-1-Financials-Proxy"]/section/div[4]/div[1]/div[1]/div[2]') %>%
          .[[1]] %>% html_children() %>% html_nodes("div")
        suppressWarnings(Values <- p[sapply(p, function(x){length(x %>% html_attrs())}) == 3] %>% 
                           html_text() %>% gsub(pattern = ",",replacement = "") %>% gsub(pattern = "-",replacement = 0) %>% as.numeric()) 
        Headers <- wahis.session %>% 
          html_nodes(xpath = '//*[@id="Col1-1-Financials-Proxy"]/section/div[4]/div[1]/div[1]') %>% 
          .[[1]] %>% html_nodes("span") %>% html_text() %>% head(5)
        Concepts <- p[sapply(p, function(x){length(x %>% html_attrs())}) == 2] %>% html_text() %>% .[seq(2,175,4)]
        BS <- data.frame(Breakdown = Concepts[-c(1:3,11,12,22:24,32,39)], matrix(Values[!is.na(Values)], ncol = 4, byrow = T))
        names(BS) <- Headers
        temp2 <- BS
        
        # Cash flows
        # cash_f <- paste0(url,stock[i],"/cash-flow?p=",stock[i])
        # wahis.session <- html_session(cash_f)
        # p <-    wahis.session %>% 
        #   html_nodes(xpath = '//*[@id="Col1-1-Financials-Proxy"]/section/div[4]/div[1]/div[1]/div[2]') %>%
        #   .[[1]] %>% html_children() %>% html_nodes("div")
        # suppressWarnings(Values <- p[sapply(p, function(x){length(x %>% html_attrs())}) == 3] %>% 
        #                    html_text() %>% gsub(pattern = ",",replacement = "") %>% gsub(pattern = "-",replacement = 0) %>% as.numeric())
        # Headers <- wahis.session %>% 
        #   html_nodes(xpath = '//*[@id="Col1-1-Financials-Proxy"]/section/div[4]/div[1]/div[1]') %>% 
        #   .[[1]] %>% html_nodes("span") %>% html_text() %>% head()
        # Concepts <- p[sapply(p, function(x){length(x %>% html_attrs())}) == 2] %>% html_text() %>% .[seq(2,133,4)]
        # CF <- data.frame(Breakdown = Concepts[-c(1,13,20,30)], matrix(Values[!is.na(Values)], ncol = 5, byrow = T))
        # names(CF) <- Headers
        # temp3 <- CF
        
        # Shares Outstanding
        stock_url <- paste0(url,stock[i],"/key-statistics?p=",stock[i])
        SO <- html_session(stock_url) %>%
          html_nodes(xpath = '//*[@id="Col1-0-KeyStatistics-Proxy"]/section/div[2]/div[2]/div/div[2]/div/table/tbody/tr[3]/td[2]') %>% 
          html_text() %>% gsub(pattern = "[A-z]",replacement = "") %>% as.numeric()
        temp4 <- SO
        
        assign(paste0(stock[i],'.f'),value = list(IS = temp1,BS = temp2,CF = temp3, SO = temp4),envir = parent.frame())
        
      },
      error = function(cond){
        message(stock[i], "Give error ",cond)
      }
    )
  }
}

#stock=Symbols
#scrapy_stocks(Symbols) 
#attach(WALMEX.MX.f)
#WALMEX.MX.f$BS
#WALMEX.MX.f$IS
#WALMEX.MX.f$CF
#BS

stqt

In [ ]:
%%R
TSE <- "Total stockholders' equity"
TL <- "Total Liabilities"
# dt <- colnames(BS)[2] # ÚLTIMA FECHA DISPONIBLE

#Fecha de valuación
dt="12/31/2021"

#Total stockholders' equity en millones
# K_0=BS[BS$Breakdown==TSE, dt]*1000/1000000
K_0=1531691*1000/1000000

#Total Liabilities DEUDA ACTUAL en millones
D_0=8569467*1000/1000000

#Shares Outstanding (Sexpresa en miles de millones en la página) en millones
Numeroacciones=stqt$Numeroacciones/1000000

dt=strptime(dt, "%m/%d/%Y")
dt=format(dt, "%Y-%m")

n=nrow(stock_prices[paste("/",dt,sep="")]) #NUMERO DE FILAS A ANALIZAR PARA EL ÚLTIMO DATO
A_0=stock_prices[paste("/",dt,sep="")][n] #valores actuales

print("precio actual de la acción")
print(A_0)

A_0=A_0*Numeroacciones #VALOR ACTUAL DE LAS ACCIONES VALOR DE ACCIÓN POR ACCIONES DISPONIBLES


A=stock_prices[paste("/",dt,sep="")]  #VECTOR DE PRECIOS 

vol_A=sqrt(var(log(as.matrix(A[2:n])/as.matrix(A[1:(n-1)]))))*sqrt(252) #VOLATILIDAD ANUAL

#Valor del CETE
r=0.0715
T=1


print("Ratio entre A0 y K0")
print(A_0/K_0)


In [ ]:
%%R
x=c(D_0,vol_A)

#Función para estimar vol_k y D
G = function(x) 
{
  vol_k <- x[2]
  D <- x[1]
  c(A_0-K_0*pnorm((log(K_0/D)+(r+0.5*vol_k^2)*T)/(vol_k*sqrt(T)))  
  +D*exp(-r*T)*pnorm((log(K_0/D)+(r-0.5*vol_k^2)*T)/(vol_k*sqrt(T))),
  A_0*vol_A-vol_k*K_0*pnorm((log(K_0/D)+(r+0.5*vol_k^2)*T)/(vol_k*sqrt(T))) )
}

ss=multiroot(G,x,positive = TRUE) #RESOLVER EL SISTEMA.

D=ss$root[1]
vol_k=ss$root[2]

print("D/D_0")
print(ss$root[1]/D_0)
print("Vol_A/Vol_K")
print(vol_A/ss$root[2])
print("evaluacion en raíces")
print(G(c(D,vol_k)))

#Probabilidad de incumplimiento
PD_merton=1-pnorm((log(K_0/D)+(r+0.5*vol_k^2)*T)/(vol_k*sqrt(T)))
print("PD MERTON")
PD_merton

In [ ]:
%%R
#Sólo estimar la volatilidad dada la deuda
#Probabilidad de Incumplimiento

Vk = function(x) 
{
  vol_k <- x
  ((vol_k/vol_A)*(K_0/A_0))/(pnorm((log(K_0/1)+(r+0.5*vol_k^2)*T)/(vol_k*sqrt(T))))
}

#Definir una Deuda
D1=D_0
x=vol_A #semilla de la vol_k

ss=multiroot(Vk,x,positive = TRUE) #RESOLVER EL SISTEMA.


print("vol_k")
vol_k=ss$root
print(vol_k)
print("Ratio vol_k/vol_A")
print(ss$root[1]/vol_A)
# print("valor Vk bajo solución")
# Vk(vol_k)

PD_merton=1-pnorm((log(K_0/D_0)+(r-0.5*vol_k^2)*T)/(vol_k*sqrt(T)))

print("Deuda sencilla")
print(D_0)
print("Vol estimada")
vol_k
print("Vol semilla")
vol_A
print("PD Merton")
PD_merton

In [ ]:
%%R
#KMV
#Función para estimar vol_k y D
Gkmv = function(x) 
{
  vol_k <- x[2]
  K_0 <- x[1]
  c(A_0-K_0*pnorm((log(K_0/D)+(r+0.5*vol_k^2)*T)/(vol_k*sqrt(T)))+D*exp(-r*T)*pnorm((log(K_0/D)+(r-0.5*vol_k^2)*T)/(vol_k*sqrt(T))),
    A_0*vol_A-vol_k*K_0*pnorm((log(K_0/D)+(r+0.5*vol_k^2)*T)/(vol_k*sqrt(T))) )
}

D=D_0

x=c(K_0,vol_A)

ss=multiroot(Gkmv,x,positive = TRUE) #RESOLVER EL SISTEMA.

K_e=ss$root[1]
vol_k=ss$root[2]
# print("Cociente K_0/D_0")
K_e/D_0
# print("Cociente vol_K/vol_A")
ss$root[2]/vol_A
# print("Valor de solución en función:")
Gkmv(c(K_e,vol_k))

DI=(K_e-D)/(vol_k*K_0)
print("DI")
print(DI)
EDF=pnorm(-DI)
print("EDF Normal")
EDF


En resumen, para la acción de Mercado Libre se tiene:
$$K_0=1,531,691,000$$ 
$$D_0=8,569,467,000$$
$$\sigma_A=0.4363978$$

Con una probabilidad de incumplimiento
$$P= 0$$
cuando la deuda y la volatilidad son estimadas, pero cuando se calcula con la deuda igual a $D_0$ se obtiene la siguiente probabilidad de Merton
$$P=1$$
este valor se puede deber a que la deuda es casi 8 veces el valor de la empresa al tiempo $t=0$.

(Los valores pueden cambiar debido a la fecha de valuación.)
